In [1]:
import numpy as np
import os
import sys
import warnings
import time
# Define paths to relevant modules
measure_sampling_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Measure_sampling')
aot_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/AOT_numerics')
trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Trees')
Benchmark_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Benchmark_value_Gausian')
awd_trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/AWD_trees')

# Add paths to sys.path
for path in [measure_sampling_path, aot_path, awd_trees_path, trees_path]:
    if path not in sys.path:
        sys.path.append(path)

# Import necessary modules
from Gen_Path_and_AdaptedTrees import *
from mainfunctions import *
from measure import *
from normal_ot import *
from FVI_bench import *

from Gen_Path_and_AdaptedTrees import generate_adapted_tree
from Tree_Node import *
from TreeAnalysis import *
from TreeVisualization import *
from Save_Load_trees import *
from Tree_AWD_utilities import *
from Gurobi_AOT import *
from Nested_Dist_Algo import compute_nested_distance
from Build_trees_from_paths import *

# Suppress sklearn warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
# ----- Parameters -----
num_paths = 10          # Number of sample paths for each measure.
time_steps = 4          # Total time steps (each path will have time_steps+1 entries, including time 0).
scale_x = 1.0 
scale_y = 4             # Scale parameter for Brownian motion.
use_weights = 1          # Set to 1 to compute weights via empirical clustering.

# ----- Generate Sample Paths and Empirical Measures -----
# Generate Brownian motion samples for μ and ν.
sample_paths_x, sample_time_y = generate_brownian_motion(num_paths, time_steps, scale_x, return_time=True)
sample_paths_y, sample_time_Y = generate_brownian_motion(num_paths, time_steps, scale_y, return_time=True)

# Compute empirical measures using k-means clustering.
# The empirical_k_means_measure function returns a tuple: (support array, weights).
new_sample_paths_x, new_weights_x = empirical_k_means_measure(sample_paths_x, use_weights=use_weights)
new_sample_paths_y, new_weights_y = empirical_k_means_measure(sample_paths_y, use_weights=use_weights)

print("Empirical Measure for μ (support):", new_sample_paths_x.shape)
print("Empirical Measure for ν (support):", new_sample_paths_y.shape)

# ----- Build the Graph -----
# We assume the columns of new_sample_paths_x and new_sample_paths_y correspond to time steps.
T_h = new_sample_paths_x.shape[1]  # Number of time steps in the empirical measure.
g = Graph(T_h)
for t in range(T_h - 1):
    g.addEdge(t, t + 1)

# ----- Define the Measure and Support Functions -----
# For a static (non-adapted) measure, we can simply ignore the conditioning.
def mu(node, x_parents):
    # Return the marginal at the given time (node) as a 2D array (support) and the weights.
    # Here, new_sample_paths_x is assumed to be of shape (k, T_h) where k is the number of clusters.
    support = new_sample_paths_x[:, node:node+1]  # Extract column 'node' as a (k,1) array.
    return [support, new_weights_x]

def supp_mu(node_list):
    # Given a list of node indices, return the corresponding columns from new_sample_paths_x.
    if len(node_list) == 0:
        # If there are no nodes, return an empty array with the right number of rows.
        return np.empty((new_sample_paths_x.shape[0], 0))
    return new_sample_paths_x[:, node_list]

def nu(node, x_parents):
    support = new_sample_paths_y[:, node:node+1]
    return [support, new_weights_y]

def supp_nu(node_list):
    if len(node_list) == 0:
        return np.empty((new_sample_paths_y.shape[0], 0))
    return new_sample_paths_y[:, node_list]

# ----- Define the Squared Cost Function -----
# This cost function computes the squared difference.
def square_cost(x, y):
    return (x[0] - y[0])**2

# Build the cost list for solve_dynamic.
# For each time node t, we associate a cost function that depends only on that node.
cost_list = [[[t], square_cost] for t in range(T_h)]

# ----- Solve the Adapted Optimal Transport Problem -----
start_time = time()
# solve_dynamic returns a list of optimal values and additional coupling information.
out_vals, opt_info = solve_dynamic(cost_list, mu, nu, supp_mu, supp_nu, g, outputflag=0, method='pot')
elapsed_time = time() - start_time

# ----- Display the Results -----
print("Optimal dynamic transport values:", out_vals)
print("Elapsed time (seconds):", elapsed_time)

Empirical Measure for μ (support): (7, 4)
Empirical Measure for ν (support): (8, 4)
Set parameter Username
Set parameter LicenseID to value 2604970
Academic license - for non-commercial use only - expires 2026-01-03
Optimal dynamic transport values: [35.59824978371171]
Elapsed time (seconds): 0.5004169940948486


In [3]:
def extract_sample_paths(mu, T, init):
    """
    Extract full sample paths and their weights from a disintegrated measure.
    
    Parameters:
      mu: A function representing the disintegrated measure. It accepts (node, x_parents)
          and returns a tuple [support, weights], where support is a 2D array (n x 1) of possible
          values at the given node and weights is a list (or array) of probabilities.
      T: Total number of time steps (i.e., nodes are 0, 1, ..., T). Note that T is the final node.
      init: The initial state (used at node 0).
    
    Returns:
      paths: A NumPy array of shape (N_paths, T+1) where each row is a sample path.
      weights: A NumPy array of shape (N_paths,) with the corresponding probability weights.
    """
    # Start at node 0 with the initial state.
    paths = [[init]]
    weights = [1.0]
    
    # For nodes 1 through T, extend each path using the measure mu.
    for t in range(1, T + 1):
        new_paths = []
        new_weights = []
        # For each path constructed so far:
        for path, w in zip(paths, weights):
            # For a Markov measure, we pass the last state as the only parent.
            parent = path[-1]
            # Get the disintegrated measure at node t.
            support, supp_weights = mu(t, [parent])
            # Each row in support is assumed to be an array of shape (1,); extract its value.
            for s, p in zip(support, supp_weights):
                # s is a 1D array (of length 1); extract its scalar value.
                new_val = s[0]
                new_paths.append(path + [new_val])
                new_weights.append(w * p)
        paths = new_paths
        weights = new_weights

    return np.array(paths), np.array(weights)

np.random.seed(12345)

N_INSTANCE = 5
T = 10  # number of non-trivial time-steps (time 0 starting at 0 is not included)
N_BRANCH = 2
x_vol = 1.0
y_vol = 0.5
x_init = 0.0
y_init = 0.0

def cost(x, y):
    return (x[0] - y[0]) ** 2

final_result_bw = np.zeros(N_INSTANCE)
final_result_pot = np.zeros(N_INSTANCE)

g = Graph(T + 1)  # indices of the graph are 0, 1, 2
# Markovian structure:
for t in range(T):
    g.addEdge(t, t + 1)

bw_times = []
pot_times = []

t0 = time()

for n_ins in range(N_INSTANCE):
    print('Instance:', n_ins)
    mu, supp_mu = rand_tree_binom(T, init=x_init, vol=x_vol, N_leaf=N_BRANCH, in_size=200)
    nu, supp_nu = rand_tree_binom(T, init=y_init, vol=y_vol, N_leaf=N_BRANCH, in_size=200)

    sample_path_x, weight_x = extract_sample_paths(mu, T, x_init)
    sample_path_y, weight_y = extract_sample_paths(nu, T, y_init)

    x_root = build_tree_from_paths(sample_path_x, weight_x)
    y_root = build_tree_from_paths(sample_path_y, weight_y)

    # Compute optimal transport using POT solver
    print("\nComputing adapted optimal transport using POT solver...")
    max_depth = get_depth(x_root)
    start_time_pot = time()
    distance_pot = compute_nested_distance(
        x_root, y_root, max_depth, method="solver_pot", return_matrix=False, lambda_reg=0, power=2
    )
    elapsed_time_pot = time() - start_time_pot
    pot_times.append(elapsed_time_pot)
    final_result_pot[n_ins] = distance_pot

    print(f"POT Solver Distance: {distance_pot:.4f}, Computation Time: {elapsed_time_pot:.2f}s")

    # Backward induction
    cost_funs = [[[t], cost] for t in range(T + 1)]
    start_time_bw = time()
    BW_v1, _ = solve_dynamic(cost_funs, mu, nu, supp_mu, supp_nu, g, outputflag=0, method='pot')
    elapsed_time_bw = time() - start_time_bw
    bw_times.append(elapsed_time_bw)

    BW_v1 = BW_v1[0] - (x_init - y_init) ** 2

    print('Values for Backward induction bicausal:', BW_v1)
    final_result_bw[n_ins] = BW_v1

# Calculating mean and standard deviation for final results and times
t_bc_2 = time() - t0

mean_bw = np.mean(final_result_bw)
std_bw = np.std(final_result_bw)
mean_pot = np.mean(final_result_pot)
std_pot = np.std(final_result_pot)

mean_bw_time = np.mean(bw_times)
mean_pot_time = np.mean(pot_times)

print('All final BW values:', final_result_bw)
print('Final BW mean:', mean_bw)
print('Final BW std:', std_bw)
print('Average time for BW computation:', mean_bw_time)

print('All final POT values:', final_result_pot)
print('Final POT mean:', mean_pot)
print('Final POT std:', std_pot)
print('Average time for POT computation:', mean_pot_time)

Instance: 0

Computing adapted optimal transport using POT solver...
POT Solver Distance: 13.2737, Computation Time: 100.99s
Values for Backward induction bicausal: 13.273737018173524
Instance: 1

Computing adapted optimal transport using POT solver...
POT Solver Distance: 11.8773, Computation Time: 100.09s
Values for Backward induction bicausal: 11.877348065861604
Instance: 2

Computing adapted optimal transport using POT solver...
POT Solver Distance: 11.7457, Computation Time: 110.08s
Values for Backward induction bicausal: 11.74569571692007
Instance: 3

Computing adapted optimal transport using POT solver...
POT Solver Distance: 10.5855, Computation Time: 103.15s
Values for Backward induction bicausal: 10.585450571724596
Instance: 4

Computing adapted optimal transport using POT solver...
POT Solver Distance: 11.9132, Computation Time: 99.00s
Values for Backward induction bicausal: 11.913016713521163
All final BW values: [13.27373702 11.87734807 11.74569572 10.58545057 11.91301671]

Instance: 0
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 19.557369051474844
sampling time 19.676290035247803
algo time 74.32159805297852
Instance: 1
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 17.43699583388584
sampling time 113.74139213562012
algo time 75.19637298583984
Instance: 2
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 18.383574076400635
sampling time 207.9128613471985
algo time 74.41569685935974
Instance: 3
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 17.16732241717519
sampling time 301.4160451889038
algo time 74.40986895561218
Instance: 4
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 18.451879608405477
sampling time 394.7682490348816
algo time 74.84406208992004
Instance: 5
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 21.26195481405158
sampling time 489.8927471637726
algo time 74.99895215034485
Instance: 6
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 17.662429183205372
sampling time 583.0186710357666
algo time 75.42059397697449
Instance: 7
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 20.497035763104357
sampling time 680.0778880119324
algo time 78.2066011428833
Instance: 8
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 19.045259487710958
sampling time 777.7145190238953
algo time 75.98750805854797
Instance: 9
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 21.08086313417913
sampling time 874.1771812438965
algo time 74.24289798736572
All final value: [19.55736905 17.43699583 18.38357408 17.16732242 18.45187961 21.26195481
 17.66242918 20.49703576 19.04525949 21.08086313]
Final mean: 19.05446833695934
Final std: 1.422736428234532
Average time for LP bicausal 94.84201941490173


Empirical Measure for μ (support): (19, 4)
Empirical Measure for ν (support): (25, 4)
Set parameter Username
Set parameter LicenseID to value 2604970
Academic license - for non-commercial use only - expires 2026-01-03
Optimal dynamic transport values: [79.83700691520171]
Elapsed time (seconds): 25.37927007675171
Instance: 0
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 19.55736905147484
sampling time 19.02067995071411
algo time 24.735156059265137
Instance: 1
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 17.436995833885838
sampling time 62.66698384284973
algo time 25.383909940719604
Instance: 2
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 18.383574076400627
sampling time 109.51078200340271
algo time 25.36733388900757
Instance: 3
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 17.167322417175185
sampling time 156.30804800987244
algo time 24.374594926834106
Instance: 4
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 18.45187960840547
sampling time 199.53148198127747
algo time 24.604914903640747
Instance: 5
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 21.261954814051578
sampling time 243.0863959789276
algo time 24.48851990699768
Instance: 6
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 17.662429183205365
sampling time 288.0112419128418
algo time 24.40409803390503
Instance: 7
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 20.497035763104353
sampling time 335.9748878479004
algo time 24.80251717567444
Instance: 8
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 19.045259487710954
sampling time 382.3044970035553
algo time 24.913800954818726
Instance: 9
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified
Values for Backward induction bicausal 21.080863134179125
sampling time 429.6232099533081
algo time 24.583000898361206
All final value: [19.55736905 17.43699583 18.38357408 17.16732242 18.45187961 21.26195481
 17.66242918 20.49703576 19.04525949 21.08086313]
Final mean: 19.05446833695933
Final std: 1.4227364282345327
Average time for LP bicausal 45.420631885528564

Instance: 0
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified

Computing adapted optimal transport using POT solver...
POT Solver Distance: 20.5574, Computation Time: 23.88s
Values for Backward induction bicausal: 19.55736905147484
Instance: 1
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified

Computing adapted optimal transport using POT solver...
POT Solver Distance: 18.4370, Computation Time: 23.96s
Values for Backward induction bicausal: 17.436995833885838
Instance: 2
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified

Computing adapted optimal transport using POT solver...
POT Solver Distance: 19.3836, Computation Time: 23.97s
Values for Backward induction bicausal: 18.383574076400627
Instance: 3
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified

Computing adapted optimal transport using POT solver...
POT Solver Distance: 18.1673, Computation Time: 24.21s
Values for Backward induction bicausal: 17.167322417175185
Instance: 4
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified

Computing adapted optimal transport using POT solver...
POT Solver Distance: 19.4519, Computation Time: 23.42s
Values for Backward induction bicausal: 18.45187960840547
Instance: 5
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified

Computing adapted optimal transport using POT solver...
POT Solver Distance: 22.2620, Computation Time: 24.94s
Values for Backward induction bicausal: 21.261954814051578
Instance: 6
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified

Computing adapted optimal transport using POT solver...
POT Solver Distance: 18.6624, Computation Time: 23.78s
Values for Backward induction bicausal: 17.662429183205365
Instance: 7
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified

Computing adapted optimal transport using POT solver...
POT Solver Distance: 21.4970, Computation Time: 23.70s
Values for Backward induction bicausal: 20.497035763104353
Instance: 8
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified

Computing adapted optimal transport using POT solver...
POT Solver Distance: 20.0453, Computation Time: 23.62s
Values for Backward induction bicausal: 19.045259487710954
Instance: 9
Warning: You are using a measure where only one-step supports are specified
Warning: You are using a measure where only one-step supports are specified

Computing adapted optimal transport using POT solver...
POT Solver Distance: 22.0809, Computation Time: 23.99s
Values for Backward induction bicausal: 21.080863134179125
All final BW values: [19.55736905 17.43699583 18.38357408 17.16732242 18.45187961 21.26195481
 17.66242918 20.49703576 19.04525949 21.08086313]
Final BW mean: 19.05446833695933
Final BW std: 1.4227364282345327
Average time for BW computation: 51.62701573371887
All final POT values: [20.55736905 18.43699583 19.38357408 18.16732242 19.45187961 22.26195481
 18.66242918 21.49703576 20.04525949 22.08086313]
Final POT mean: 20.05446833695933
Final POT std: 1.422736428234533
Average time for POT computation: 23.94796974658966